<a href="https://colab.research.google.com/github/RLWH/reinforcement-learning-notebook/blob/master/4.%20Model%20Free%20Control/Ch5_7(c)_Off_policy_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# On-policy learning vs Off-policy learning

On-policy learning is about learning the most optimal behaviour through the policy that the agent is following. However, the agent needs to behave non-optimally in order to explore all actions to find the optimal actions. So, how can they learn about th eoptimal policy while behaving according to an exploratory policy?

An alternative way to learn the most optimal policy is to use two policies
- One policy is going to learn the most optimal behaviour - called the *target policy $\pi$*
- Another policy is more exploratory and is used to generate behaviour - called the *behaviour/exploratory policy $\mu$*

Since in this case we say that learning is from data "off" the target policy, thus the overall process is termed *off-policy* learning

## Pros and cons of On-policy learning and Off-policy learning

##### On-policy
- Concept is simpler
- Easier to converge

##### Off-policy
- Concept is harder
- slower to converge
- Greater variance
- More exploration
- Learn from others' experience
- Key to learning multi-step predictive models of the world's dynamics

## The prediction problem of off-policy methods

Suppose we wish to estimate $v_\pi$ or $q_\pi$, but all we have are episodes folowing another policy $\mu$, where $\mu \neq \pi$. Under this setting, we call that the policy $\pi$ is the target policy, and this is the policy that we want to learn and optimise. The policy $\mu$ is the behaviour policy, where we will sample the actions from. Both policies are considered fixed and given.

##### Some requirements - The assumption of converge
If we want to use episodes from $\mu$ to estimate values for $\pi$, we need to ensure that every action taken from $\pi$, is at least occasionally taken under $\mu$. Formally, we require $\pi(a|s) > 0 \implies \mu(a|s) > 0$. 

Under this setting, the policy $\mu$ must be stochastic in states where it is not identical to $\mu$, while the policy $\mu$ can be deterministic. For simplicity, we can assume policy $\mu$ is a greedy policy. i.e. $\text{argmax}_a \pi(a|s) = 1$

### The cornerstone of off-policy methods - Importance Sampling
Importance sampling is a general technique for estimating expected values under one distribution given samples from another. It weights the returns of each timestep according to the relative probability of their trajectories occuring under the target and behaviour policies. 

Given a starting state $S_t$, 